In [5]:
from deap import base, creator, tools
from rdkit import RDLogger
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import Lipinski
from ml_models.classifier import Classifier
from tqdm import tqdm
import pandas as pd
import numpy as np
import random

In [6]:
THEO_SMILES = 'N1C=NC2=C1C(=O)N(C)C(=O)N2C'
data = pd.read_csv('../data/database_CCDC.csv', header=None)
data.columns = ['smiles']
data.head()

,smiles
0,CC1(C)C(=O)NC1S(=O)(=O)C1=CC=CC=C1
1,CC1C(=O)NC1S(=O)(=O)C1=CC=CC=C1
2,OC1=CC=CC(O)=C1
3,C(=CC1=CC=NC=C1)C1=CC=NC=C1
4,C1=NC=CC(C2C(C3=CC=NC=C3)C(C3=CC=NC=C3)C2C2=CC...


### Классы для дополниельных оценок

In [7]:
class SARCalculator:
    """Класс для оценки сложности синтеза молекул (SAR) по шкале 0-1"""

    def __init__(self):
        self.MIN_SAR = 1.0
        self.MAX_SAR = 5.0

    def _estimate_synthesis_steps(self, mol):
        return min(5, max(1, int(mol.GetNumAtoms() / 10)))

    def _estimate_reaction_conditions(self, mol):
        if THEO_SMILES in Chem.MolToSmiles(mol):
            return 0.1  # Базовый теофиллин - простые условия
        return min(1.0, Descriptors.NumRadicalElectrons(mol) * 0.2 + Descriptors.NumRotatableBonds(mol) * 0.05)

    def _predict_yield(self, mol):
        complexity_factor = Descriptors.MolLogP(mol) / 10
        return max(0.1, 1.0 - complexity_factor)

    def _chromatography_complexity(self, mol):
        return min(1.0, Descriptors.NumRotatableBonds(mol) * 0.15 + Descriptors.TPSA(mol) * 0.005)

    def calculate_normalized_SAR(self, smiles):
        mol = Chem.MolFromSmiles(smiles)
        if not mol:
            return 0.0

        components = {
            'steps': self._estimate_synthesis_steps(mol),
            'conditions': self._estimate_reaction_conditions(mol),
            'yield': self._predict_yield(mol),
            'chromatography': self._chromatography_complexity(mol)
        }

        weights = [0.4, 0.3, 0.2, 0.1]
        raw_sar = np.dot(weights, [
            components['steps'],
            components['conditions'] * 5,
            5 * (1 - components['yield']),
            components['chromatography'] * 5
        ])

        normalized_sar = (raw_sar - self.MIN_SAR) / (self.MAX_SAR - self.MIN_SAR)
        return max(0.0, min(1.0, normalized_sar))

In [8]:
class CocrystalEvaluator:
    """Класс для оценки ко-кристаллизации теофиллина (шкала 0-1)"""

    def __init__(self):
        self.theo_smiles = THEO_SMILES
        self.theo_mol = Chem.MolFromSmiles(self.theo_smiles)

    def _calculate_h_bond_score(self, cof_mol):
        h_donors = min(Descriptors.NumHDonors(cof_mol), 2)
        h_acceptors = min(Descriptors.NumHAcceptors(self.theo_mol), 3)
        return (h_donors / 2 * 0.6) + (h_acceptors / 3 * 0.4)

    def _calculate_functional_groups(self, cof_mol):
        score = 0
        if cof_mol.HasSubstructMatch(Chem.MolFromSmarts('C(=O)O')):
            score += 0.5  # Карбоксил
        if cof_mol.HasSubstructMatch(Chem.MolFromSmarts('[OH]')):
            score += 0.3  # Гидроксил
        if cof_mol.HasSubstructMatch(Chem.MolFromSmarts('[NH2]')):
            score += 0.2  # Аминогруппа
        return score

    def _calculate_stereo_match(self, cof_mol):
        delta_logp = abs(Descriptors.MolLogP(self.theo_mol) - Descriptors.MolLogP(cof_mol))
        return max(0, 1.0 - delta_logp / 5)  # Нормировка разницы LogP

    def calculate_cocrystal_score(self, cof_smiles):
        cof_mol = Chem.MolFromSmiles(cof_smiles)
        if not cof_mol:
            return 0.0

        components = {
            'h_bonds': self._calculate_h_bond_score(cof_mol),
            'func_groups': self._calculate_functional_groups(cof_mol),
            'stereo': self._calculate_stereo_match(cof_mol)
        }

        return 0.4 * components['h_bonds'] + 0.3 * components['func_groups'] + 0.3 * components['stereo']

In [ ]:
sar_calculator = SARCalculator()
cocrystal_evaluator = CocrystalEvaluator()

### Функции кроссовера и мутации

In [9]:
RDLogger.DisableLog('rdApp.*')


def calculate_complexity(smiles):
    try:
        mol = Chem.MolFromSmiles(smiles)
        return Descriptors.MolComplexity(mol) if mol else None
    except:
        return 100


def calculate_druglike_solubility(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if not mol:
        return None

    logp = Descriptors.MolLogP(mol)                     # Липофильность (чем меньше, тем лучше)
    tpsa = Descriptors.TPSA(mol)                        # Полярная поверхность (чем больше, тем лучше)
    mw = Descriptors.MolWt(mol)                         # Молекулярный вес
    h_donors = Lipinski.NumHDonors(mol)                 # Доноры водорода
    h_acceptors = Lipinski.NumHAcceptors(mol)           # Акцепторы водорода
    rotatable_bonds = Lipinski.NumRotatableBonds(mol)   # Подвижные связи

    lipinski_flags = [
        logp <= 5,
        mw <= 500,
        h_donors <= 5,
        h_acceptors <= 10,
        rotatable_bonds <= 10
    ]
    lipinski_score = sum(lipinski_flags) / len(lipinski_flags)

    solubility_score = (
            0.4 * (1 - max((logp - 1) / 4, 0) +
                   0.3 * (tpsa / 140) +
                   0.2 * (1 - mw / 500) +
                   0.1 * lipinski_score)
    )

    return max(0, min(1, solubility_score))


def calculate_tabletability(smiles_str, drug=THEO_SMILES):
    properties_to_predict = ['unobstructed', 'orthogonal_planes', 'h_bond_bridging']

    classification = Classifier()

    properties = classification.predict_properties(drug, smiles_str, properties_to_predict)
    lst = list(properties.values())

    sar_score = sar_calculator.calculate_normalized_SAR(smiles_str)
    score = calculate_druglike_solubility(smiles_str)

    total_score = sar_score
    return lst[0], lst[1], (1 - lst[2]), total_score, score


def crossover(ind1, ind2):
    min_length = 2

    if len(ind1) < min_length or len(ind2) < min_length:
        return ind1, ind2

    max_point = min(len(ind1), len(ind2)) - 1
    if max_point < 1:
        return ind1, ind2

    point = random.randint(1, max_point)

    ind1_new = creator.Individual(ind1[:point] + ind2[point:])
    ind2_new = creator.Individual(ind2[:point] + ind1[point:])

    return ind1_new, ind2_new

In [10]:
FUNCTIONAL_GROUPS = [
    # H-доноры (приоритет для таблетируемости)
    {"smiles": "C(=O)O", "weight": 2.3, "h_bond": True},
    {"smiles": "O", "weight": 1, "h_bond": True},
    {"smiles": "N", "weight": 1.2, "h_bond": True},
    {"smiles": "C(=O)N", "weight": 1.1, "h_bond": True},

    # Галогены (пониженный вес)
    {"smiles": "F", "weight": 0.3, "h_bond": False},
    {"smiles": "Cl", "weight": 0.2, "h_bond": False},

    # Потенциально токсичные (минимальный вес)
    {"smiles": "N(=O)=O", "weight": 0.1, "h_bond": False}]


def is_valid_molecule(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if not mol:
        return False
    atom_types = [atom.GetSymbol() for atom in mol.GetAtoms()]
    for i in range(len(atom_types) - 1):
        if (atom_types[i] == "N" and atom_types[i + 1] == "N") or (atom_types[i] == "O" and atom_types[i + 1] == "O"):
            return False
    return True

def is_cyclic(mol):
    return mol.GetRingInfo().NumRings() > 0

def count_atoms(mol):
    atom_counts = {
        "N": 0,
        "O": 0,
        "F": 0,
        "Br": 0,
        "Cl": 0
    }
    for atom in mol.GetAtoms():
        symbol = atom.GetSymbol()
        if symbol in atom_counts:
            atom_counts[symbol] += 1
    return atom_counts

def mutate(ind):
    mol = Chem.MolFromSmiles(ind)
    if not mol or len(ind) > 50:
        return ind

    for _ in range(10):  # Попробуем выполнить мутацию до 10 раз
        mutation_type = random.choices(
            ["replace_atom", "add_group"],
            weights=[0.4, 0.6],
            k=1
        )[0]

        if mutation_type == "replace_atom":
            atom_idx = random.randint(0, mol.GetNumAtoms() - 1)
            new_atom = random.choices(
                ["C", "N", "O"],
                weights=[0.4, 0.2, 0.1],
                k=1
            )[0]
            if atom_idx < mol.GetNumAtoms() - 1 and ind[atom_idx+1] == "C" and new_atom in ["N", "O"]:
                mol.GetAtomWithIdx(atom_idx).SetAtomicNum(Chem.GetPeriodicTable().GetAtomicNumber(new_atom))
                new_smiles = Chem.MolToSmiles(mol)
            elif new_atom == "C":
                mol.GetAtomWithIdx(atom_idx).SetAtomicNum(Chem.GetPeriodicTable().GetAtomicNumber(new_atom))
                new_smiles = Chem.MolToSmiles(mol)
            else:
                continue

        else:
            valid_groups = [g for g in FUNCTIONAL_GROUPS]
            weights = [g["weight"] for g in valid_groups]

            total_weight = sum(weights)
            normalized_weights = [w / total_weight for w in weights]

            group = random.choices(valid_groups, weights=normalized_weights, k=1)[0]["smiles"]

            if is_cyclic(mol):
                cyclic_atoms = [atom.GetIdx() for atom in mol.GetAtoms() if atom.IsInRing()]
                non_cyclic_atoms = [i for i in range(mol.GetNumAtoms()) if i not in cyclic_atoms]
                if non_cyclic_atoms:
                    index = random.choice(non_cyclic_atoms)
                else:
                    continue
            else:
                index = random.randint(0, len(ind))
            if group == "O" and is_cyclic(mol):
                continue
                
            # Проверка на добавление галогена
            if index > 0 and ind[index-1] == "C":
                new_smiles = ind[:index] + group + ind[index:]
            else:
                continue

        if not is_valid_molecule(new_smiles):
            continue

        new_mol = Chem.MolFromSmiles(new_smiles)

        # Проверка на количество колец и их размеры
        if not new_mol or new_mol.GetNumAtoms() > 35 or any(len(ring) > 6 for ring in new_mol.GetRingInfo().AtomRings()):
            continue

        # Проверка на количество атомов
        atom_counts = count_atoms(new_mol)
        if (atom_counts["N"] > 4 or atom_counts["O"] > 4 or
                atom_counts["F"] > 4 or atom_counts["Br"] > 2 or
                atom_counts["Cl"] > 2):
            continue

        return creator.Individual(new_smiles)

    return ind

In [11]:
creator.create("FitnessMulti", base.Fitness, weights=(1.0, 1.0, 1.0, 1.0, 1.0))
creator.create("Individual", str, fitness=creator.FitnessMulti)

toolbox = base.Toolbox()

initial_smiles = data['smiles'].tolist()
toolbox.register("smiles", random.choice, initial_smiles)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.smiles)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", lambda ind: calculate_tabletability(ind))
toolbox.register("mate", crossover)
toolbox.register("mutate", mutate)
toolbox.register("select", tools.selTournament, tournsize=3)

population = toolbox.population(n=30)
N_GEN = 30
CXPB = 0.6
MUTPB = 0.2

### Эволюционная оптимизация

In [ ]:
for gen in tqdm(range(N_GEN)):
    offspring = toolbox.select(population, len(population))
    offspring = list(map(toolbox.clone, offspring))

    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            child1, child2 = toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for i in range(len(offspring)):
        if random.random() < MUTPB:
            offspring[i] = toolbox.mutate(offspring[i])
            del offspring[i].fitness.values

    # Удаление дубликатов с принудительной мутацией
    seen_smiles = set()
    new_offspring = []

    for ind in offspring:
        mol = Chem.MolFromSmiles(ind)
        if mol is None:
            continue
        can_smiles = Chem.MolToSmiles(mol, canonical=True)

        if can_smiles not in seen_smiles:
            seen_smiles.add(can_smiles)
            new_offspring.append(ind)
        else:
            mutated = None
            attempts = 0
            max_attempts = 100
            while attempts < max_attempts:
                mutated_candidate = toolbox.mutate(ind)
                mol_mut = Chem.MolFromSmiles(mutated_candidate)
                if mol_mut is None:
                    attempts += 1
                    continue
                new_smiles = Chem.MolToSmiles(mol_mut, canonical=True)
                if new_smiles not in seen_smiles:
                    mutated = creator.Individual(mutated_candidate)
                    seen_smiles.add(new_smiles)
                    break
                attempts += 1

            if mutated:
                new_offspring.append(mutated)

    offspring = new_offspring

    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    valid_ind = list(filter(lambda ind: Chem.MolFromSmiles(ind) is not None, invalid_ind))
    fitnesses = map(toolbox.evaluate, valid_ind)

    for ind, fit in zip(valid_ind, fitnesses):
        ind.fitness.values = fit

    population[:] = offspring
    print(gen, population)

#### Финальный отбор

In [ ]:
valid_population = [ind for ind in population if Chem.MolFromSmiles(ind) is not None and ind.fitness.valid]
top = tools.selBest(valid_population, 1000)

print("\n🔝 Топ молекул по tabletability score:\n")
for i, ind in enumerate(top, 1):
    print(f"{i}. SMILES: {ind}, Tabletability Score: {ind.fitness.values}")

#### Изображения полученных молекул

In [ ]:
Chem.MolFromSmiles(valid_population[0])

In [ ]:
Chem.MolFromSmiles(valid_population[1])

In [ ]:
Chem.MolFromSmiles(valid_population[10])

In [ ]:
Chem.MolFromSmiles(valid_population[20])

In [ ]:
Chem.MolFromSmiles(valid_population[60])